Real Time Radar Parameter Optimizer
===================================

Carson Anderson & Calvin Henggeler  
ASEN 5264 Decision Making Under Uncertainty - Spring 2025  
Semester Project  

In [8]:
using POMDPs
using POMDPTools: transition_matrices, reward_vectors, SparseCat, Deterministic, RolloutSimulator, DiscreteBelief, FunctionPolicy, ordered_states, ordered_actions, DiscreteUpdater, UnderlyingMDP
using QuickPOMDPs: QuickPOMDP
using NativeSARSOP: SARSOPSolver
using POMDPTesting: has_consistent_distributions
using QMDP: QMDPSolver
using Plots
using Statistics: std
using POMDPPolicies: alphavectors, FunctionPolicy
using Random, Distributions
include("radarSimulator.jl")

radar_meas_2_obs_state (generic function with 1 method)

## 1. Create Smart Radar POMDP

In [ ]:
# radar_tracking_calvin = QuickPOMDP(
#     
#     # --- Environment Parameters ---
#     c = 3*10^8  # Speed of light
#     f = 1e9     # Frequency
#     λ = c/f     # Wavelength
#     x_coverage_area = 100e3     # 100 km x 100 km area
#     y_coverage_area = 100e3
#     radar_location = (50e3, 0)  # Radar location (x, y) in meters
# 
#     # --- STATE SPACE ---
#     states = nothing,     # We'll model state generation implicitly
#     
#     # --- ACTION SPACE ---
#     actions = [
#         # Example discretized radar settings (fill in realistic values)
#         # start with a range of differeent realistic settings
#         (beamwidth=5, pulse_length=0.000001, Tipp= 0.001) # , rotation_rate=5),
#         (beamwidth=5, pulse_length=0.000001, Tipp= 0.001) # , rotation_rate=10),
#         (beamwidth=5, pulse_length=0.000002, Tipp= 0.001) # , rotation_rate=5),
#         (beamwidth=5, pulse_length=0.000002, Tipp= 0.001) # , rotation_rate=10),
#     ],
#     # 
#     
#     obstype = Vector{Float64},   # Observation: [power, phase, delay, doppler]
# 
#     gen = function (s, a, rng)
#         # s = (x, y, vx, vy),   extract sate information
#         x, y, vx, vy = s
#         true_range = sqrt((xp - radar_location[1])^2 + (yp - radar_location[2])^2)
# 
#         #TODO: set noise parameters based on radar settings, then add noise to obsercations measurments
# 
# 
#         # Observation Measurments
#         received_power  = radar_return_power(λ, true_range)
#         view_angle      = radar_view_angle(x, y, radar_location)
#         delay           = radar_return_delay(true_range)
#         doppler_obs     = radar_doppler(s)
#         measurments = [received_power, view_angle, delay, doppler_obs]
# 
#         # solve for the observed state
#         o = radar_meas_2_obs_sate()
# 
#         # REWARD: - (tracking error) - (velocity_error) - (action cost)
#         xo, yo, vxo, vyo = o
#         tracking_error = sqrt((x - xo)^2 + (y - yo)^2)              # Euclidean distance
#         velocity_error = sqrt(vx^2 + vy^2) - sqrt(vxo^2 + vyo^2)    # difference of absolute velocities
#         r = -tracking_error - velocity_error
#         # peanalties for actions should realistically be seen in greater measurement noise
# 
#         # TRANSITION MODEL: Target motion (simple linear)
#         dt = T_ipp * 10     # time step
#         xp = x + vx*dt      # + noise_std*randn(rng)
#         yp = y + vy*dt      # + noise_std*randn(rng)
#                             # keep velocities
#         sp = (xp, yp, vx, vy)
# 
#         return (sp=sp, o=o, r=r)
#     end,
# 
#     # TINITIAL STATE GENERATION
#     # Target starts at either far side edge of the coverage area (x ∈ [0, 100e3], y ∈ [0, 100e3])
#     # target will aways start at y = 50e3, ending at a random y at the other side, gives relative heading
#     # Absolute velocity is uniformy distributed from 50 - 300 m/s, v_x and v_x derived from heading and absolute velocity
#     
#     function generate_initial_target(rng::AbstractRNG=Random.default_rng())
#         # 1. Start on left or right edge
#         start_x = rand(rng, [0.0, 100e3])  # 0 or 100,000 m
#     
#         # 2. Always start at y = 50 km
#         start_y = 50e3  # meters
#     
#         # 3. Pick random end y on the opposite side
#         end_y = rand(rng) * 100e3  # [0, 100_000] m
#     
#         # 4. Compute heading angle from start to end
#         dx = (start_x == 0.0) ? 100e3 : -100e3
#         dy = end_y - start_y
#         heading_rad = atan(dy, dx)  # angle in radians
#     
#         # 5. Choose absolute speed [50, 300] m/s
#         speed = rand(rng, Uniform(50.0, 300.0))
#     
#         # 6. Derive velocity components
#         vx = speed * cos(heading_rad)
#         vy = speed * sin(heading_rad)
#     
#         return (x=start_x, y=start_y, vx=vx, vy=vy)b
#     end
#     
#     initiatial_state generate_initial_target()
# 
#     discount = 0.99,
# 
#     isterminal = s -> false  # Tracking problem — no natural terminal state 
#     # TODO: termainal states will be when the target reaches the end of the radar simulator
# )


In [ ]:
radar = QuickPOMDP(
    

    # --- STATE SPACE ---
    states = nothing,     # We'll model state generation implicitly
    
    # --- ACTION SPACE ---
    actions = [:Search, :Guess], #third action to change parameters of radar settings?
    # 
    
    obstype = Vector{Float64},   # Observation: [steering_angle, range, peak_power]

    observation = function(s, a)
        if target near action direction (some angle width)
            detected = true
            range = true_range + randn()*10.0
            angle = action_angle + randn()*2.0
            strength = 0.8 + randn()*0.1
        else
            detected = false
            range = NaN
            angle = NaN
            strength = 0.0
        end
        return (detected, range, angle, strength)

    end,

    reward = function (s,a)
        if a == :Search
            return -1.0  # Cost of searching
        elseif 
    end 

    # TINITIAL STATE GENERATION
    # Target starts at either far side edge of the coverage area (x ∈ [0, 100e3], y ∈ [0, 100e3])
    # target will aways start at y = 50e3, ending at a random y at the other side, gives relative heading
    # Absolute velocity is uniformy distributed from 50 - 300 m/s, v_x and v_x derived from heading and absolute velocity
    
    # function generate_initial_target(rng::AbstractRNG=Random.default_rng())
    #     # 1. Start on left or right edge
    #     start_x = rand(rng, [0.0, 100e3])  # 0 or 100,000 m
    # 
    #     # 2. Always start at y = 50 km
    #     start_y = 50e3  # meters
    # 
    #     # 3. Pick random end y on the opposite side
    #     end_y = rand(rng) * 100e3  # [0, 100_000] m
    # 
    #     # 4. Compute heading angle from start to end
    #     dx = (start_x == 0.0) ? 100e3 : -100e3
    #     dy = end_y - start_y
    #     heading_rad = atan(dy, dx)  # angle in radians
    # 
    #     # 5. Choose absolute speed [50, 300] m/s
    #     speed = rand(rng, Uniform(50.0, 300.0))
    # 
    #     # 6. Derive velocity components
    #     vx = speed * cos(heading_rad)
    #     vy = speed * sin(heading_rad)
    # 
    #     return (x=start_x, y=start_y, vx=vx, vy=vy)
    # end
    #
    # initiatial_state generate_initial_target()

    initial_state = (x=50e3, y=50e3),  # Initial state of the target

    discount = 0.99,

    isterminal = s -> false  # Tracking problem — no natural terminal state 
    # TODO: termainal states will be when the target reaches the end of the radar simulator
)


## 2. Create Updater (Particle Filter)